In [1]:
import numpy as np
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

from tqdm.auto import tqdm

import wandb

import logging
import sys

c:\conda-envs\envs\DLankle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def setup_logger(name=__name__):
    """
    Sets up a logger that outputs to the console (stdout).
    """
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler(sys.stdout)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

logger = setup_logger()

In [3]:
# Load the dataset
data_folder = "../data"
preped_folder = os.path.join(data_folder, "_preped")

train_data = pd.read_csv(os.path.join(data_folder, 'train_data.csv')).values.tolist()
test_data = pd.read_csv(os.path.join(data_folder, 'test_data.csv')).values.tolist()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to consistent size
    transforms.ToTensor(),           # Convert to tensor [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

x_train = []
y_train = []

for img_name, label in train_data:
    img_path = os.path.join(preped_folder, img_name)
    try:
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)
        x_train.append(img_tensor)
        y_train.append(label)
    except Exception as e:
        logger.info(f"Error loading {img_name}: {e}")

# Stack into tensors
x_train_tensor = torch.stack(x_train)
logger.info(f"Training images shape: {x_train_tensor.shape}")

# Encode labels to integers
label_to_idx = {label: idx for idx, label in enumerate(np.unique(y_train))}
y_train_encoded = [label_to_idx[label] for label in y_train]
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)

logger.info(f"Training labels shape: {y_train_tensor.shape}")
logger.info(f"Label mapping: {label_to_idx}")

2025-12-07 11:36:10,037 - INFO - Training images shape: torch.Size([221, 3, 224, 224])
2025-12-07 11:36:10,038 - INFO - Training labels shape: torch.Size([221])
2025-12-07 11:36:10,039 - INFO - Label mapping: {np.str_('1_Pronacio'): 0, np.str_('2_Neutralis'): 1, np.str_('3_Szupinacio'): 2}


In [4]:
x_test = []
y_test = []

for img_name, label in test_data:
    img_path = os.path.join(preped_folder, img_name)
    try:
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)
        x_test.append(img_tensor)
        y_test.append(label)
    except Exception as e:
        logger.info(f"Error loading {img_name}: {e}")

x_test_tensor = torch.stack(x_test)
logger.info(f"Test images shape: {x_test_tensor.shape}")
y_test_encoded = [label_to_idx[label] for label in y_test]
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

logger.info(f"Test labels shape: {y_test_tensor.shape}")

2025-12-07 11:36:17,386 - INFO - Test images shape: torch.Size([49, 3, 224, 224])
2025-12-07 11:36:17,387 - INFO - Test labels shape: torch.Size([49])


In [5]:
if torch.cuda.is_available():
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    logger.info(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        logger.info(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        logger.info(f"  Memory: {props.total_memory / 1024**3:.2f} GB")
        logger.info(f"  Compute Capability: {props.major}.{props.minor}")
else:
    logger.info("CUDA not available")

2025-12-07 11:36:17,441 - INFO - CUDA available: True
2025-12-07 11:36:17,448 - INFO - Number of GPUs: 1
2025-12-07 11:36:17,456 - INFO - 
GPU 0: NVIDIA GeForce RTX 4060
2025-12-07 11:36:17,461 - INFO -   Memory: 8.00 GB
2025-12-07 11:36:17,462 - INFO -   Compute Capability: 8.9


In [6]:
batch_size = 32
num_epochs = 15
lr=0.0001
device = 'cuda' 

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# wandb login an init
# Login to wandb with API key
load_dotenv()
wandb.login(key=os.getenv("wandbKey"))

def init_wandb():
    # Initialize wandb project
    wandb.init(
        project="ankle-align",
        config={
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "learning_rate": lr,
            "architecture": "Custom CNN",
            "dataset": "AnkleAlign",
            "optimizer": "Adam"
        }
    )


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Win 10\_netrc
wandb: Currently logged in as: bencefarkas (bencefarkas-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model = None
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
            self.counter = 0


In [9]:
def train_model(network, optimizer, loss_fn, enable_early_stopping=False, patience=5):
    torch.cuda.empty_cache()

    loss_values = []

    if enable_early_stopping:
        early_stopping = EarlyStopping(patience=patience, verbose=True)

    network.train()
    for epoch in tqdm(range(num_epochs), desc='Training model'):
        network.train()
        epoch_loss = 0.0
        num_batches = 0
        for images, target_labels in train_loader:
            images = images.to(device)
            target_labels = target_labels.to(device)

            pred_logits = network(images)
            loss = loss_fn(pred_logits, target_labels)
            epoch_loss += loss.item()
            num_batches += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = epoch_loss / num_batches

        if enable_early_stopping:
            network.eval()
            val_loss = 0.0
            val_batches = 0
            correct = 0
            total = 0
            with torch.no_grad():
                for images, target_labels in val_loader:
                    images = images.to(device)
                    target_labels = target_labels.to(device)
                    
                    pred_logits = network(images)
                    loss = loss_fn(pred_logits, target_labels)
                    val_loss += loss.item()
                    val_batches += 1
                    
                    _, predicted = torch.max(pred_logits, 1)
                    total += target_labels.size(0)
                    correct += (predicted == target_labels).sum().item()
            
            avg_val_loss = val_loss / val_batches
            val_accuracy = correct / total

        # Log metrics
        if enable_early_stopping:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss,
                "val_accuracy": val_accuracy
            })
        else:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss
            })
        
        
        if enable_early_stopping:
            logger.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
        else:
            logger.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")

        # Early stopping check
        if enable_early_stopping:
            early_stopping(avg_val_loss, network)
            if early_stopping.early_stop:
                logger.info("Early stopping triggered")
                network.load_state_dict(early_stopping.best_model)
                break
    
    # Load best model
    if enable_early_stopping and early_stopping.best_model is not None:
        network.load_state_dict(early_stopping.best_model)
        logger.info("Loaded best model weights")

In [10]:
def evaluate_model(network):
    # Training score
    true_labels = y_test_encoded
    predicted_labels = []
    network.eval()
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            outputs = network(images)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = np.mean([true == pred for true, pred in zip(true_labels, predicted_labels)])
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    logger.info(f"network accuracy: {accuracy * 100:.2f}%")
    logger.info(f"network precision: {precision * 100:.2f}%")
    logger.info(f"network recall: {recall * 100:.2f}%")
    logger.info(f"network F1 score: {f1 * 100:.2f}%")

    logger.info(f"Detailed Classification Report: \n{classification_report(true_labels, predicted_labels)}")

    # Log test metrics
    wandb.log({
        "test_accuracy": accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "test_f1": f1
    })

    wandb.finish()

In [11]:
net1 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # 224x224 -> 112x112
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(64 * 56 * 56, 128),  # 64 channels * 56 * 56 spatial size
    torch.nn.ReLU(),
    torch.nn.Linear(128, 3)  # 3 classes output
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net1.parameters(), lr=lr)

In [12]:
init_wandb()
train_model(net1, optimizer, loss_fn)
evaluate_model(net1)

Training model:   0%|          | 0/15 [00:00<?, ?it/s]

2025-12-07 11:36:22,859 - INFO - Epoch 1/15, Train Loss: 1.2294


Training model:   7%|▋         | 1/15 [00:00<00:06,  2.03it/s]

2025-12-07 11:36:23,018 - INFO - Epoch 2/15, Train Loss: 0.8834


Training model:  13%|█▎        | 2/15 [00:00<00:03,  3.38it/s]

...

2025-12-07 11:36:25,111 - INFO - network accuracy: 55.10%



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


2025-12-07 11:36:25,112 - INFO - network precision: 89.29%
2025-12-07 11:36:25,112 - INFO - network recall: 55.10%
2025-12-07 11:36:25,113 - INFO - network F1 score: 64.22%
2025-12-07 11:36:25,121 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.25      0.71      0.37         7
           1       1.00      0.52      0.69        42
           2       0.00      0.00      0.00         0

    accuracy                           0.55        49
   macro avg       0.42      0.41      0.35        49
weighted avg       0.89      0.55      0.64        49



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
wandb: ERROR The nbformat package was not foun

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▆▅▄▄▃▃▃▂▂▂▁▁▁▁
epoch,15
test_accuracy,0.55102
test_f1,0.6422
test_precision,0.89286
test_recall,0.55102


In [13]:
# Use train test split, for getting validation metrics during training
x_train_tensor, x_val_tensor, y_train_tensor, y_val_tensor = train_test_split(
    x_train_tensor, y_train_tensor, test_size=0.2, random_state=42, stratify=y_train_tensor)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [14]:
net1 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # 224x224 -> 112x112
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(64 * 56 * 56, 128),  # 64 channels * 56 * 56 spatial size
    torch.nn.ReLU(),
    torch.nn.Linear(128, 3)  # 3 classes output
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net1.parameters(), lr=lr)

In [15]:
init_wandb()
train_model(net1, optimizer, loss_fn, enable_early_stopping=True)
evaluate_model(net1)

Training model:   0%|          | 0/15 [00:00<?, ?it/s]

2025-12-07 11:36:28,697 - INFO - Epoch 1/15, Train Loss: 1.2682, Val Loss: 1.1586, Val Acc: 0.4889


Training model:   7%|▋         | 1/15 [00:00<00:06,  2.07it/s]

2025-12-07 11:36:28,857 - INFO - Epoch 2/15, Train Loss: 0.9279, Val Loss: 1.0025, Val Acc: 0.6222


Training model:  13%|█▎        | 2/15 [00:00<00:03,  3.42it/s]

...

2025-12-07 11:36:29,921 - INFO - Loaded best model weights
2025-12-07 11:36:29,952 - INFO - network accuracy: 44.90%
2025-12-07 11:36:29,952 - INFO - network precision: 90.48%
2025-12-07 11:36:29,953 - INFO - network recall: 44.90%



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


2025-12-07 11:36:29,953 - INFO - network F1 score: 52.26%
2025-12-07 11:36:29,964 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         7
           1       1.00      0.36      0.53        42
           2       0.00      0.00      0.00         0

    accuracy                           0.45        49
   macro avg       0.44      0.45      0.34        49
weighted avg       0.90      0.45      0.52        49



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
wandb: ERROR The nbformat package was not foun

epoch,▁▂▃▄▅▅▆▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▆▄▄▃▂▂▁▁
val_accuracy,▁█▂█▇▆▆▇▇
val_loss,█▄▁▁▁▁▂▂▃
epoch,9
test_accuracy,0.44898
test_f1,0.52256


In [16]:
net1 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # 224x224 -> 112x112
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(64 * 56 * 56, 128),  # 64 channels * 56 * 56 spatial size
    torch.nn.ReLU(),
    torch.nn.Linear(128, 3)  # 3 classes output
).to(device)


def init_weights(m):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)

net1.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net1.parameters(), lr=lr)

In [17]:
init_wandb()
train_model(net1, optimizer, loss_fn, enable_early_stopping=True)
evaluate_model(net1)
# Weight initialization with Kaiming did not improve the results, probably due to small network, 
# we might get back to it later.

Training model:   0%|          | 0/15 [00:00<?, ?it/s]

2025-12-07 11:36:33,071 - INFO - Epoch 1/15, Train Loss: 5.9141, Val Loss: 4.7751, Val Acc: 0.2444


Training model:   7%|▋         | 1/15 [00:00<00:04,  3.00it/s]

2025-12-07 11:36:33,223 - INFO - Epoch 2/15, Train Loss: 2.8750, Val Loss: 2.8247, Val Acc: 0.5111


Training model:  13%|█▎        | 2/15 [00:00<00:02,  4.42it/s]

...

2025-12-07 11:36:34,444 - INFO - Loaded best model weights
2025-12-07 11:36:34,472 - INFO - network accuracy: 40.82%



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


2025-12-07 11:36:34,473 - INFO - network precision: 87.14%
2025-12-07 11:36:34,474 - INFO - network recall: 40.82%
2025-12-07 11:36:34,475 - INFO - network F1 score: 55.08%
2025-12-07 11:36:34,483 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.10      0.14      0.12         7
           1       1.00      0.45      0.62        42
           2       0.00      0.00      0.00         0

    accuracy                           0.41        49
   macro avg       0.37      0.20      0.25        49
weighted avg       0.87      0.41      0.55        49



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
wandb: ERROR The nbformat package was not foun

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▄▃▂▁▁▁▁▁▁
val_accuracy,▁▆▇▇▇█▇▇█▆
val_loss,█▄▂▁▁▁▁▁▁▁
epoch,10
test_accuracy,0.40816
test_f1,0.55076


In [18]:
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # 224x224 -> 112x112
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56
    torch.nn.ReLU(),
    torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # 56x56 -> 28x28
    torch.nn.ReLU(),
    torch.nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # 28x28 -> 14x14
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(256 * 14 * 14, 128),  # 256 channels * 14 * 14 spatial size
    torch.nn.ReLU(),
    torch.nn.Linear(128, 3)  # 3 classes output
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net2.parameters(), lr=lr)

In [19]:
init_wandb()
train_model(net2, optimizer, loss_fn, enable_early_stopping=True)
evaluate_model(net2)

Training model:   0%|          | 0/15 [00:00<?, ?it/s]

2025-12-07 11:36:37,326 - INFO - Epoch 1/15, Train Loss: 1.0519, Val Loss: 1.0187, Val Acc: 0.4889


Training model:   7%|▋         | 1/15 [00:00<00:04,  3.27it/s]

2025-12-07 11:36:37,470 - INFO - Epoch 2/15, Train Loss: 0.9819, Val Loss: 0.9923, Val Acc: 0.4889


Training model:  13%|█▎        | 2/15 [00:00<00:02,  4.75it/s]

...

2025-12-07 11:36:38,568 - INFO - Loaded best model weights
2025-12-07 11:36:38,600 - INFO - network accuracy: 24.49%
2025-12-07 11:36:38,601 - INFO - network precision: 88.57%
2025-12-07 11:36:38,601 - INFO - network recall: 24.49%
2025-12-07 11:36:38,602 - INFO - network F1 score: 28.95%



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


2025-12-07 11:36:38,612 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.20      0.71      0.31         7
           1       1.00      0.17      0.29        42
           2       0.00      0.00      0.00         0

    accuracy                           0.24        49
   macro avg       0.40      0.29      0.20        49
weighted avg       0.89      0.24      0.29        49



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▇▆▆▅▄▃▂▂▁
val_accuracy,▁▁▄▃▂▁▄▂▅█
val_loss,█▆▅▃▁▂▃▃▆▃
epoch,10
test_accuracy,0.2449
test_f1,0.28954


In [ ]:
net3 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 224x224 -> 112x112
    torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 112x112 -> 56x56
    torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 56x56 -> 28x28
    torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 28x28 -> 14x14
    torch.nn.Flatten(),
    torch.nn.Linear(256 * 14 * 14, 128),  # 256 channels * 14 * 14 spatial size
    torch.nn.ReLU(),
    torch.nn.Linear(128, 3)  # 3 classes output
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net3.parameters(), lr=lr)

In [ ]:
init_wandb()
train_model(net3, optimizer, loss_fn, enable_early_stopping=True, patience=3)
evaluate_model(net3)

Training model:   0%|          | 0/15 [00:00<?, ?it/s]

2025-12-07 11:57:48,096 - INFO - Epoch 1/15, Train Loss: 0.9977, Val Loss: 1.0593, Val Acc: 0.4889


Training model:   7%|▋         | 1/15 [00:00<00:09,  1.54it/s]

2025-12-07 11:57:48,476 - INFO - Epoch 2/15, Train Loss: 1.0117, Val Loss: 1.0036, Val Acc: 0.5333


Training model:  13%|█▎        | 2/15 [00:01<00:06,  2.03it/s]

...

2025-12-07 11:57:52,554 - INFO - Epoch 14/15, Train Loss: 0.5088, Val Loss: 1.0741, Val Acc: 0.5556
2025-12-07 11:57:52,582 - INFO - EarlyStopping counter: 5 out of 5
2025-12-07 11:57:52,583 - INFO - Early stopping triggered


Training model:  87%|████████▋ | 13/15 [00:05<00:00,  2.53it/s]

2025-12-07 11:57:52,586 - INFO - Loaded best model weights
2025-12-07 11:57:52,695 - INFO - network accuracy: 30.61%
2025-12-07 11:57:52,695 - INFO - network precision: 88.28%
2025-12-07 11:57:52,696 - INFO - network recall: 30.61%
2025-12-07 11:57:52,696 - INFO - network F1 score: 31.78%
2025-12-07 11:57:52,707 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.18      1.00      0.30         7
           1       1.00      0.19      0.32        42
           2       0.00      0.00      0.00         0

    accuracy                           0.31        49
   macro avg       0.39      0.40      0.21        49
weighted avg       0.88      0.31      0.32        49




c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\

epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,██▇▆▆▅▅▄▃▃▂▂▁▁
val_accuracy,▁▃▃▂▅▁▅▅▃█▅▅▃▅
val_loss,▆▄▃▂▅▂▂▂▁▃▃▅█▆
epoch,14
test_accuracy,0.30612
test_f1,0.31776


In [ ]:
lr=0.00001

net4 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    torch.nn.BatchNorm2d(32),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 224x224 -> 112x112

    torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    torch.nn.BatchNorm2d(64),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 112x112 -> 56x56

    torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    torch.nn.BatchNorm2d(128),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 56x56 -> 28x28

    torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    torch.nn.BatchNorm2d(256),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),  # 28x28 -> 14x14
    
    torch.nn.Flatten(),
    torch.nn.Linear(256 * 14 * 14, 128),  # 256 channels * 14 * 14 spatial size
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 3)  # 3 classes output
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net3.parameters(), lr=lr)

In [47]:
init_wandb()
train_model(net4, optimizer, loss_fn, enable_early_stopping=True, patience=3)
evaluate_model(net4)

Training model:   0%|          | 0/15 [00:00<?, ?it/s]

2025-12-07 12:27:18,398 - INFO - Epoch 1/15, Train Loss: 1.0946, Val Loss: 1.0878, Val Acc: 0.3556


Training model:   7%|▋         | 1/15 [00:00<00:08,  1.73it/s]

2025-12-07 12:27:18,795 - INFO - Epoch 2/15, Train Loss: 1.1182, Val Loss: 1.0779, Val Acc: 0.4889


Training model:  13%|█▎        | 2/15 [00:00<00:06,  2.12it/s]

...

2025-12-07 12:27:20,740 - INFO - Epoch 7/15, Train Loss: 1.1372, Val Loss: 1.0892, Val Acc: 0.4667
2025-12-07 12:27:20,740 - INFO - EarlyStopping counter: 3 out of 3
2025-12-07 12:27:20,740 - INFO - Early stopping triggered


Training model:  40%|████      | 6/15 [00:02<00:04,  2.05it/s]

2025-12-07 12:27:20,744 - INFO - Loaded best model weights
2025-12-07 12:27:20,802 - INFO - network accuracy: 24.49%
2025-12-07 12:27:20,803 - INFO - network precision: 60.53%
2025-12-07 12:27:20,803 - INFO - network recall: 24.49%
2025-12-07 12:27:20,804 - INFO - network F1 score: 30.05%
2025-12-07 12:27:20,813 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.08      0.43      0.14         7
           1       0.69      0.21      0.33        42

    accuracy                           0.24        49
   macro avg       0.39      0.32      0.23        49
weighted avg       0.61      0.24      0.30        49




wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▂▃▅▆▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,▁▅▄▁▇▇█
val_accuracy,▁▆██▇█▅
val_loss,█▅▂▁▃▆█
epoch,7
test_accuracy,0.2449
test_f1,0.30045
